In [1]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas
sia = SentimentIntensityAnalyzer()
print("done")

done


In [9]:
orig_df = pandas.read_json('tweet_reply.json', lines=True)
# orig_df.info()

In [11]:
orig_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170255 entries, 0 to 170254
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   keyword      170255 non-null  object
 1   main_tweet   170255 non-null  object
 2   main_likes   170255 non-null  int64 
 3   reply        170255 non-null  object
 4   reply_likes  170255 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 6.5+ MB


In [12]:
orig_df.head()

,keyword,main_tweet,main_likes,reply,reply_likes
0,COVID-19,this entire bioweapon coup d'etat ~ their Covi...,3,I’ve had enough tbh!,45
1,COVID-19,Some ppl pretend to remain ignorant. There was...,0,"Resp PM Ji,Pl rate yourself on target defined",0
2,COVID-19,Some ppl pretend to remain ignorant. There was...,0,#ModiDisasterForIndia Irony is that he still i...,1
3,COVID-19,Some ppl pretend to remain ignorant. There was...,0,With terms &amp; conditions no natural calamit...,0
4,COVID-19,Some ppl pretend to remain ignorant. There was...,0,All of them have been fulfilled,0


In [13]:
orig_df.to_numpy()

array([['COVID-19',
        "this entire bioweapon coup d'etat ~ their Covid-19 'Great Reset' that took out small biz, forced jabs to keep an income, shredded family relationships, killed people in ICU's w/ Fauci Death protocols - firing squad would seem legit.",
        3, 'I’ve had enough tbh!', 45],
       ['COVID-19',
        'Some ppl pretend to remain ignorant. There was Covid19 that crippled world economy for almost 2 yrs. India was no exception. The comeback becomes all the more important. Need to read real financial &amp; economic news/analysis than being blinded by hatred towards a person or party.',
        0, 'Resp PM Ji,Pl rate yourself on target defined', 0],
       ['COVID-19',
        'Some ppl pretend to remain ignorant. There was Covid19 that crippled world economy for almost 2 yrs. India was no exception. The comeback becomes all the more important. Need to read real financial &amp; economic news/analysis than being blinded by hatred towards a person or party.',
    

In [20]:
orig_df = orig_df.sample(frac=1).reset_index(drop=True) # shuffle records

In [21]:
orig_df.head()

,keyword,main_tweet,main_likes,reply,reply_likes
0,Tesla,#tesla would be my option not perfume,0,Everyone wants to know how many perfumes have ...,0
1,Ukraine,Ukraine has got to be one of the richest count...,1,Loł Middle East must mind controlling Europe.S...,0
2,COVID-19,You mean the lies about how wearing a mask is ...,8,*any conservative,6
3,Bitcoin,Looks like your deleting your twitter. Have fu...,0,Start to delete your account,0
4,World Cup,"Penaldo 2026 WC winner aka ""the oldest footbal...",0,"C.Ronaldo , right?",0


In [22]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(orig_df, test_size=0.4)

In [24]:
test["reply"].values.size

68102

In [25]:
train["reply"].values.size

102153

In [26]:
test.head()

,keyword,main_tweet,main_likes,reply,reply_likes
163648,weather,It's those damn crazy eyes again.,0,The eyes… it’s always the eyes.,1
151263,World Cup,No. World cup isn’t the measurement to be a go...,0,There wasn't a debate to be had,1
118661,Vaccine,I know this will sound crazy…but maybe it’s ti...,4,"Reason to maintain masking, etc",0
95420,Dogecoin,DOGE Coin…,1,The love someone has for you,1
161546,Dogecoin,Elon right now...#BTC #Dogecoin,2,Musk for president 🇺🇸 #USA,0


In [28]:
test.reset_index(drop=True)

,keyword,main_tweet,main_likes,reply,reply_likes
0,weather,It's those damn crazy eyes again.,0,The eyes… it’s always the eyes.,1
1,World Cup,No. World cup isn’t the measurement to be a go...,0,There wasn't a debate to be had,1
2,Vaccine,I know this will sound crazy…but maybe it’s ti...,4,"Reason to maintain masking, etc",0
3,Dogecoin,DOGE Coin…,1,The love someone has for you,1
4,Dogecoin,Elon right now...#BTC #Dogecoin,2,Musk for president 🇺🇸 #USA,0
...,...,...,...,...,...
68097,nba,okay now there just doing what ppl want. all t...,0,If stays healthy and don’t win we have a prob...,0
68098,Elon Musk,What do this time ?,0,"Elon Musk, when will we reach 5,000 times?#Elo...",0
68099,Bitcoin,This fellow makes money in Bitcoin exchange fe...,0,If anyone had a doubt: El Salvador is a failed...,0
68100,weather,"Ur modesty is bothersome, If it was me, i woul...",0,"Welcome to the preview, er, the Pink Flaming...",6


In [30]:
train.reset_index(drop=True)

,keyword,main_tweet,main_likes,reply,reply_likes
0,Amazon,Why don't you look into Parler where this ridi...,0,BREAKING: The unselect committee will be disba...,90
1,Ukraine,Ukraine is being demilitarized slowly,0,"Deal with it, n4ziLabs: Thanks from ukr:",0
2,Bitcoin,"This is the mainstream reality we live in , th...",0,🤦🏽‍♂️ what a joke this is. Why aren’t we steal...,0
3,World Cup,"🇭🇷 In 2018, he scored twice in the group stage...",0,"🇭🇷 In 2022, he played every game as they progr...",3
4,COVID-19,Chinese and visitors from China have been posi...,2,No upstanding American is going to China,0
...,...,...,...,...,...
102148,Elon Musk,Take a trip to Moscow and stay.,0,Is this a picture of the 6 foot penis that wa...,0
102149,Dogecoin,Elon: Wanna buy when dogecoin dips (lmao),3,Everyone was betting on the freedom and indepe...,0
102150,nba,Boy needs to shoot. Simple. I'm sure Philly fa...,3,Imagine being one of the most famous basketbal...,0
102151,Amazon,Amazons are back!,0,I have to retweet this classic bang out again,0


In [33]:
test["keyword"].values[0]

'weather'

In [47]:
print(train["reply"].values[1])
print(sia.polarity_scores(train["reply"].values[1]))

print("Deal with it, n4ziLabs: Thanks from ukr:")
print(sia.polarity_scores("fuck"))


Deal with it, n4ziLabs: Thanks from ukr:
{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.4404}
Deal with it, n4ziLabs: Thanks from ukr:
{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.5423}


In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [7]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json: 100%|██████████| 747/747 [00:00<00:00, 79.8kB/s]
c:\Users\humay\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\humay\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 2.55M

In [15]:
# Run for Roberta Model
encoded_text = tokenizer("arse", return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.32950994, 'roberta_neu': 0.47112793, 'roberta_pos': 0.19936216}
